## 多输入通道

In [34]:
import torch

def corr2d(X, K):  #@save
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    # 矩阵求和 ==> 对应元素相加
    return sum(corr2d(x, k) for x, k in zip(X, K))

In [35]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

# X 输入通道数量为 2 ; 图像大小为 3 * 3
# K 卷积核大小为 2 * 2 ; 处理输入通道为 2 的图像
X.shape, K.shape

(torch.Size([2, 3, 3]), torch.Size([2, 2, 2]))

In [36]:
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 多输入和多输出通道

In [37]:
def corr2d_multi_in_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

# K.shape 输出通道, 输入通道, 图像高, 图像宽
K = torch.stack((K, K+1, K+2), 0)
K.shape  # 输出通道为 3

torch.Size([3, 2, 2, 2])

In [38]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 1*1 的卷积核

In [39]:
def corr2d_multi_in_out_1x1(X, K):
    """
    使用矩阵乘法实现 1*1 的卷积
    c_o = channel output 输出通道
    c_i = channel input 输入通道
    h, w = 图像的高, 宽
    :param X:
    :param K:
    :return:
    """
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [40]:
# X 输入通道, 高, 宽
# K 输出通道, 输入通道, 卷积核高, 宽
X = torch.normal(0, 1, (3, 3, 3))  # 均值为 0 方差为 1 形状是 (3, 3, 3)
K = torch.normal(0, 1, (2, 3, 1, 1))

# 两种方式都是 1*1 的卷积
# 验证使用矩阵乘法实现的 1*1 卷积 和 multi_in_out 实现的 1*1 卷积得到的结果是一致的
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6